In [ ]:
# Import necessary libraries
import pandas as pd, shutil, os
import yfinance as yf
import datetime as dt
import numpy as np
from pathlib import Path
import json

In [ ]:
# Get S&P 500 tickers
sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500_tickers = sp500_tickers.Symbol.to_list()
# sp500_tickers

In [ ]:
# Clean Tickers of periods
sp500_tickers = [x.replace(".", "-") for x in sp500_tickers]
# sp500_tickers

In [ ]:
# Define Start and End dates for hist. data(5 yr)
end_date = dt.datetime.now().date().isoformat()
start_date = dt.datetime.now() - dt.timedelta(weeks=26)
start_date = start_date.date().isoformat()
start_date

In [ ]:
# Removes old Stock_Data folder and replaces it with a new, empty one
shutil.rmtree("Stock_Data")
os.mkdir("Stock_Data")

In [ ]:
sp_data = yf.download(sp5_tickers, period="5d", group_by="Ticker")
sp_data

In [ ]:
csv_path = Path("Stock_Data/Stock_Data.json")

In [ ]:
stock_dict = {}

In [ ]:
sp_data_j = sp_data.to_json()

In [ ]:
type(stock_dict)

In [ ]:
type(sp_data_j)

In [ ]:
sp_data_j = json.loads(sp_data_j)
type(sp_data_j)

In [ ]:
print(json.dumps(sp_data_j, indent=4))

In [ ]:
# reading the JSON data using json.load()
# file = 'data.json'
# with open(file) as train_file:
#     dict_train = json.load(train_file)

# converting json dataset from dictionary to dataframe
train = pd.DataFrame.from_dict(sp_data_j, orient='index')
train.reset_index(level=0, inplace=True)

In [ ]:
train

In [ ]:
with open(csv_path, 'w') as outfile:
     json.dump(sp_data.to_json(), outfile, indent=4)

In [ ]:
with open(csv_path, "r") as infile:
    stock_dict = json.dump(infile)
    
stock_dict

In [ ]:
json_file = open(csv_path, "r").read()
# stock_d = json.load(json_file)
# json_file.close()
# print(json.dumps(stock_d, indent=4, sort_keys=True))
# data = urlopen(q).read()
json_data = json.loads(json_file) # DECODE THE STRING 
with open('output.json', 'w') as out:
    json.dump(json_data, out, indent=4) # DO NOT CALL STR ON IT AGAIN

print(json.dumps(out))

In [48]:
def ForwardPE():
    tick_list = []
    forwardPE = []
    count = 0
    for ticker in sp5_tickers:
        info = yf.Ticker(ticker).info
        info["Ticker"] = ticker
        forwardPE.append(info["forwardPE"])
        tick_list.append(info["Ticker"])
        count+=1
        print(count)
    pe_df = pd.DataFrame(data=forwardPE, index=tick_list, columns=["Forward P/E"])
    pe_df.sort_values(by="Forward P/E", ascending=False, inplace=True)
    
    return pe_df
# fin = yf.Ticker("MSFT").info

In [49]:
ForwardPE()

1
2
3
4
5


,Forward P/E
ABMD,61.500954
ACN,32.837563
ABT,26.521551
MMM,18.976349
ABBV,8.337867


In [ ]:
# def calcRSI (ticker):
#     data = yf.download(ticker, period="5y", groupby="Ticker")
#     data["Moving_Avg_200_Days"] = data["Adj Close"].rolling(window=200).mean()
#     data["Price_Change"] = data["Adj Close"].pct_change()
#     data["Gains"] = data["Price_Change"].apply(lambda x: x if x > 0 else 0)
#     data["Losses"] = data["Price_Change"].apply(lambda x: abs(x) if x < 0 else 0)
#     data["Avg_Up"] = data["Gains"].ewm(span=19).mean()
#     data["Avg_Down"] = data["Losses"].ewm(span=19).mean()
#     data = data.dropna()
#     data["RS"] = data["Avg_Up"]/data["Avg_Down"]
#     data["RSI"] = data["RS"].apply(lambda x: 100 - (100/(x+1)))
#     return data

In [42]:
def calcRSI (ticker):
    data = yf.download(ticker, start=start_date,end=end_date, group_by="Ticker")
    data["Ticker"] = ticker
#     data["Moving_Avg_200_Days"] = data["Adj Close"].rolling(window=200).mean()
    data["Price_Change"] = data["Adj Close"].pct_change()
#     temp1 = data['Price_Change']
    data['Gains'] = data.loc[:,"Price_Change"].apply(lambda x: x if x > 0 else 0)
#     data["Gains"] = data.iloc[:,7].apply(lambda x: x if x > 0 else 0)
    data["Losses"] = data.loc[:,"Price_Change"].apply(lambda x: abs(x) if x < 0 else 0)
#     data["Losses"] = data.iloc[:,7].apply(lambda x: abs(x) if x < 0 else 0)
#     temp2 = data["Gains"]
#     temp3 = data["Losses"]
    data["Avg_Up"] = data.loc[:,"Gains"].ewm(com=13,adjust=False, min_periods=14).mean()
#     temp4 = data["Avg_Up"]
    data["Avg_Down"] = data.loc[:, "Losses"].ewm(com=13,adjust=False, min_periods=14).mean()
#     temp5 = data["Avg_Down"]

#     temp6 = temp4/temp5
    data["RS"] = data.loc[:, "Avg_Up"]/data.loc[:,"Avg_Down"]
    data["RSI"] = data.loc[:,"RS"].apply(lambda x: 100 - (100/(x+1)))
    data = data.dropna()
#     temp7 = data["RSI"]
#     output_df = pd.DataFrame(columns=["Ticker", "RSI"])
#     output_df = output_df.append({"Ticker" : ticker,
#                                  "RSI" : temp7},
#                                 ignore_index=True)
    today = data.iloc[-1]
#     output_df = pd.DataFrame(columns=["Ticker","RSI"])
#     output_df = pd.merge(output_df,today[["Ticker", "RSI"]], on="Ticker", how="left")
    return today

In [ ]:
calcRSI(sp500_tickers[0])

In [51]:
sp5_tickers= ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']

In [ ]:
# join_data = pd.DataFrame(index="Tickers",columns="RSI")

In [54]:
def Todays_RSI_Hi():
    stock_tick_list = []
    stock_rsi_list = []
    count = 0
#     join_data = pd.DataFrame()
    for stock in sp5_tickers:
        stockRSI = calcRSI(stock)
        stock_tick_list.append(stockRSI["Ticker"])
        stock_rsi_list.append(stockRSI["RSI"])
        count+=1
        print(count)
    rsi_df_hi = pd.DataFrame(data=(stock_rsi_list), index=stock_tick_list, columns=["RSI"])
    rsi_df_hi.sort_values(by=["RSI"], ascending=False,inplace=True)

    return rsi_df_hi.head()

In [55]:
fig = Todays_RSI_Hi()

[*********************100%***********************]  1 of 1 completed
1
[*********************100%***********************]  1 of 1 completed
2
[*********************100%***********************]  1 of 1 completed
3
[*********************100%***********************]  1 of 1 completed
4
[*********************100%***********************]  1 of 1 completed
5


In [56]:
fig

,RSI
ACN,68.444327
ABT,64.502504
ABBV,54.544121
MMM,53.640731
ABMD,47.793852


In [43]:
def Todays_RSI_Low():
    stock_tick_list = []
    stock_rsi_list = []
    count = 0
#     join_data = pd.DataFrame()
    for stock in sp5_tickers:
        stockRSI = calcRSI(stock)
        stock_tick_list.append(stockRSI["Ticker"])
        stock_rsi_list.append(stockRSI["RSI"])
        count+=1
        print(count)
    rsi_df_low = pd.DataFrame(data=(stock_rsi_list), index=stock_tick_list, columns=["RSI"])
    rsi_df_low = rsi_df_low.sort_values(by=["RSI"],ascending=True)

    return rsi_df_low

In [44]:
fig2 = Todays_RSI_Low()

[*********************100%***********************]  1 of 1 completed
1
[*********************100%***********************]  1 of 1 completed
2
[*********************100%***********************]  1 of 1 completed
3
[*********************100%***********************]  1 of 1 completed
4
[*********************100%***********************]  1 of 1 completed
5


In [45]:
fig2

,RSI
ABMD,47.793852
MMM,53.640731
ABBV,54.544121
ABT,64.502504
ACN,68.444327


In [ ]:
aapl_data = yf.download('AAPL', period="1mo")
aapl_data

In [ ]:
aapl_data['Ticker'] = 'AAPL'
aapl_data

In [ ]:
wmt_data = yf.download("WMT", period="1mo")
wmt_data

In [ ]:
combined = pd.merge(aapl_data, wmt_data, how="left", on=['Date'])

In [ ]:
combined

In [ ]:
""